In [40]:
# check the pytorch installation
import torch
print(torch.cuda.is_available())       # Should be True
print(torch.backends.cudnn.enabled)    # Should be True

True
True


In [41]:
import torch.utils.data
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import os
import csv
import numpy as np
import pickle
import matplotlib
matplotlib.use("Agg")
import matplotlib.pyplot as plt
import cv2

In [42]:
# camera-ready
import sys

sys.path.append("deeplabv3")
sys.path.append("deeplabv3/model")
from deeplabv3 import DeepLabV3

sys.path.append("deeplabv3/utils")
from utils import label_img_to_color


In [43]:
work_dir = '/home/kywch/deeplabv3/'
lib_dir = work_dir + 'deeplabv3/'

# Specify your output directory
input_dir = work_dir + "input/"
output_dir = work_dir + "output/"

In [44]:
network = DeepLabV3("eval_val", project_dir="deeplabv3", 
                    pretrained_file=lib_dir+"resnet18-5c106cde.pth").cuda()

pretrained resnet, 18


In [45]:
network.load_state_dict(torch.load(lib_dir+"model_13_2_2_2_epoch_580.pth"))

<All keys matched successfully>

In [46]:
class DatasetVal(torch.utils.data.Dataset):
    def __init__(self, input_path, output_path):
        self.img_dir = input_path
        self.img_h = 1024
        self.img_w = 2048
        self.output_dir = output_path
        if not os.path.exists(output_path):
            os.makedirs(output_path)
        self.debug_dir = output_path + '/debug/'
        if not os.path.exists(self.debug_dir):
            os.makedirs(self.debug_dir)
        
        self.examples = []
        file_names = os.listdir(self.img_dir)
        for file_name in file_names:
            if file_name.lower().endswith(('.jpg', '.png', '.jpeg')):
                img_id = file_name.split('.')[0]
                img_path = self.img_dir + '/' + file_name
                example = {}
                example["img_path"] = img_path
                example["img_id"] = img_id
                self.examples.append(example)
        #print(self.examples)

        self.num_examples = len(self.examples)
        print(str(self.num_examples) + " images were identified from the input directory.")
        
    def __getitem__(self, index):
        example = self.examples[index]

        img_id = example["img_id"]
        img_path = example["img_path"]
        img = cv2.imread(img_path, -1) # (shape: (1024, 2048, 3))

        # normalize the img (with the mean and std for the pretrained ResNet):
        img = img/255.0
        img = img - np.array([0.485, 0.456, 0.406])
        img = img/np.array([0.229, 0.224, 0.225]) # (shape: (512, 1024, 3))
        img = np.transpose(img, (2, 0, 1)) # (shape: (3, 512, 1024))
        img = img.astype(np.float32)

        # convert numpy -> torch:
        img = torch.from_numpy(img) # (shape: (3, 512, 1024))
        #label_img = torch.from_numpy(label_img) # (shape: (512, 1024))
        print(img_id)
        return (img, img_id)

    def __len__(self):
        return self.num_examples


In [47]:
val_dataset = DatasetVal(input_path=input_dir, output_path=output_dir)

78 images were identified from the input directory.


In [48]:
val_dataset.examples

[{'img_path': '/home/kywch/deeplabv3/input//p1r00056.jpg',
  'img_id': 'p1r00056'},
 {'img_path': '/home/kywch/deeplabv3/input//p1r00058.jpg',
  'img_id': 'p1r00058'},
 {'img_path': '/home/kywch/deeplabv3/input//p1r00050.jpg',
  'img_id': 'p1r00050'},
 {'img_path': '/home/kywch/deeplabv3/input//p1r00029.jpg',
  'img_id': 'p1r00029'},
 {'img_path': '/home/kywch/deeplabv3/input//street2.jpg', 'img_id': 'street2'},
 {'img_path': '/home/kywch/deeplabv3/input//p1r00046.jpg',
  'img_id': 'p1r00046'},
 {'img_path': '/home/kywch/deeplabv3/input//p1r00018.jpg',
  'img_id': 'p1r00018'},
 {'img_path': '/home/kywch/deeplabv3/input//p1r00065.jpg',
  'img_id': 'p1r00065'},
 {'img_path': '/home/kywch/deeplabv3/input//p1r00060.jpg',
  'img_id': 'p1r00060'},
 {'img_path': '/home/kywch/deeplabv3/input//p1r00017.jpg',
  'img_id': 'p1r00017'},
 {'img_path': '/home/kywch/deeplabv3/input//p1r00068.jpg',
  'img_id': 'p1r00068'},
 {'img_path': '/home/kywch/deeplabv3/input//p1r00052.jpg',
  'img_id': 'p1r00052

In [49]:
val_loader = torch.utils.data.DataLoader(dataset=val_dataset,
                                         batch_size=1, shuffle=False,
                                         num_workers=1)

In [50]:
network.eval()

DeepLabV3(
  (resnet): ResNet_BasicBlock_OS8(
    (resnet): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (1): BasicBlock(
          (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn1): BatchNorm2d

In [51]:
count = 0

#produce labeled image
for step, (imgs, img_ids) in enumerate(val_loader):
    #print(img_ids)
    with torch.no_grad(): # (corresponds to setting volatile=True in all variables, this is done during inference to reduce memory consumption)
        imgs = Variable(imgs).cuda() # (shape: (batch_size, 3, img_h, img_w))
        outputs = network(imgs) # (shape: (batch_size, num_classes, img_h, img_w))
  
        ########################################################################
        # save data for visualization:
        ########################################################################
        
        outputs = outputs.data.cpu().numpy() # (shape: (batch_size, num_classes, img_h, img_w))
        pred_label_imgs = np.argmax(outputs, axis=1) # (shape: (batch_size, img_h, img_w))
        pred_label_imgs = pred_label_imgs.astype(np.uint8)

        for i in range(pred_label_imgs.shape[0]):
            if i == 0:
                pred_label_img = pred_label_imgs[i] # (shape: (img_h, img_w))
                img_id = img_ids[i]
                img = imgs[i] # (shape: (3, img_h, img_w))

                img = img.data.cpu().numpy()
                img = np.transpose(img, (1, 2, 0)) # (shape: (img_h, img_w, 3))
                img = img*np.array([0.229, 0.224, 0.225])
                img = img + np.array([0.485, 0.456, 0.406])
                img = img*255.0
                img = img.astype(np.uint8)

                pred_label_img_color = label_img_to_color(pred_label_img)
                overlayed_img = 0.35*img + 0.65*pred_label_img_color
                overlayed_img = overlayed_img.astype(np.uint8)
                #producing original predicted images
                cv2.imwrite(val_dataset.output_dir + "/" + img_id + "_pred_label.png", pred_label_img )
                
                # produce debugging info for the first 10 images
                if count < 100:
                    #producing colored predicted images
                    cv2.imwrite(val_dataset.debug_dir + "/" + img_id + "_pred_label_colorimg.png", pred_label_img_color )
                    #producing overlayed images
                    cv2.imwrite(val_dataset.debug_dir + "/" + img_id + "_overlayed.png", overlayed_img)
                

        ########################################################################
        # quick counting
        ########################################################################
        count += 1
                

p1r00056
p1r00058
p1r00050
p1r00029
street2
p1r00046
p1r00018
p1r00065
p1r00060
p1r00017
p1r00068
p1r00052
street1
p1r00034
p1r00028
p1r00074
p1r00039
p1r00067
p1r00009
p1r00066
p1r00040
p1r00016
p1r00006
p1r00049
p1r00015
p1r00011
p1r00027
p1r00019
p1r00073
p1r00055
p1r00038
p1r00004
p1r00021
p1r00033
p1r00022
p1r00037
p1r00003
p1r00005
p1r00025
p1r00044
p1r00010
p1r00062
p1r00043
p1r00048
p1r00057
p1r00013
p1r00054
p1r00026
p1r00031
p1r00035
p1r00008
p1r00020
p1r00042
p1r00032
p1r00041
p1r00076
p1r00059
p1r00014
p1r00036
p1r00053
p1r00023
p1r00030
p1r00064
p1r00047
p1r00069
p1r00024
p1r00071
p1r00072
p1r00063
p1r00075
p1r00051
p1r00007
p1r00012
p1r00070
p1r00002
p1r00061
p1r00045
p1r00001


In [52]:
# generate the summary csv file from the labeled files
import glob
output_files = glob.glob(val_dataset.output_dir + "/*pred_label.png")

In [53]:
print(val_dataset.output_dir)
print(output_files)

/home/kywch/deeplabv3/output/
['/home/kywch/deeplabv3/output/p1r00030_pred_label.png', '/home/kywch/deeplabv3/output/p1r00047_pred_label.png', '/home/kywch/deeplabv3/output/p1r00019_pred_label.png', '/home/kywch/deeplabv3/output/p1r00016_pred_label.png', '/home/kywch/deeplabv3/output/p1r00073_pred_label.png', '/home/kywch/deeplabv3/output/p1r00048_pred_label.png', '/home/kywch/deeplabv3/output/p1r00026_pred_label.png', '/home/kywch/deeplabv3/output/p1r00036_pred_label.png', '/home/kywch/deeplabv3/output/p1r00025_pred_label.png', '/home/kywch/deeplabv3/output/p1r00021_pred_label.png', '/home/kywch/deeplabv3/output/p1r00008_pred_label.png', '/home/kywch/deeplabv3/output/p1r00049_pred_label.png', '/home/kywch/deeplabv3/output/p1r00022_pred_label.png', '/home/kywch/deeplabv3/output/p1r00005_pred_label.png', '/home/kywch/deeplabv3/output/p1r00064_pred_label.png', '/home/kywch/deeplabv3/output/p1r00010_pred_label.png', '/home/kywch/deeplabv3/output/p1r00044_pred_label.png', '/home/kywch/deep

In [54]:
# deeplabv3 - cityscape labels

class_names = ['pcnt_road', 'pcnt_sidewalk', 'pcnt_building', 'pcnt_wall', 'pcnt_fence',
    'pcnt_pole', 'pcnt_traffic_light', 'pcnt_traffic_sign', 'pcnt_vegetation', 'pcnt_terrain',
    'pcnt_sky', 'pcnt_person', 'pcnt_rider', 'pcnt_car', 'pcnt_truck',
    'pcnt_bus', 'pcnt_train', 'pcnt_motorcycle', 'pcnt_bicycle', 'pcnt_unlabeled']
num_class = len(class_names)


In [55]:
# quick check

file = output_files[0]

print(os.path.basename(file).split('_pred_label.png')[0])

pred_label_img = cv2.imread(file, -1)
# count how many pixels in pred_label_img which are of object class trainId:
pixel_ratio = np.zeros(num_class)
for cate_id in range(num_class):
    pixel_ratio[cate_id] = np.sum(np.equal(pred_label_img, cate_id))/pred_label_img.size

print(pixel_ratio.tolist())

p1r00030
[0.23655598958333332, 0.002802734375, 0.031119791666666667, 0.0, 0.044638671875, 0.009479166666666667, 0.000322265625, 0.0080859375, 0.4121875, 0.09728841145833333, 0.06376627604166667, 0.0, 0.0, 0.09059895833333333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.003154296875]


In [20]:
#save category ratios to csv file
with open(val_dataset.output_dir + '/00_object_ratios.csv', 'w') as output:
    writer = csv.writer(output)
    writer.writerow(['image_name'] + class_names)
    
    for file in output_files:
        img_name = os.path.basename(file).split('_pred_label.png')[0]
        pred_label_img = cv2.imread(file, -1)
        
        # count how many pixels in pred_label_img which are of object class trainId:
        pixel_ratio = np.zeros(num_class)
        for cate_id in range(num_class):
            pixel_ratio[cate_id] = np.sum(np.equal(pred_label_img, cate_id))/pred_label_img.size
        
        # write to the csv file
        writer.writerow([img_name] + pixel_ratio.tolist())
